In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
from itertools import combinations

from time import time 
from create_feature import * 

In [2]:
feature_dicts = {
        'prices': ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"],
        'sizes':  ["matched_size", "bid_size", "ask_size", "imbalance_size"],
        "category": ["stock_id", "seconds_in_bucket", 'imbalance_buy_sell_flag']
    }

csv_train = "/home/lishi/projects/Competition/kaggle_2023/data/train.csv"
df = pd.read_csv(csv_train)
df = df[~df['target'].isnull()] 

print(df.shape)
print(f"Trading days: {df['date_id'].nunique()}")
print(f"Stocks: {df['stock_id'].nunique()}")

(5237892, 17)
Trading days: 481
Stocks: 200


In [3]:
df, v1_features = gen_v1_features(df, feature_dicts['prices'])

In [4]:
v2_feat_cols = feature_dicts['prices'] + feature_dicts['sizes'] + v1_features

df, v2_features = gen_v2_features(df, v2_feat_cols)

In [5]:
group_by_stock = df.groupby(['date_id', 'stock_id'])

In [9]:
pct_features = ['imbalance_with_flag', 'wap', 'matched_imbalance']
relative_change = group_by_stock[pct_features].pct_change(1, fill_method=None).add_prefix('pct_')
df_v3 = pd.concat([df, relative_change], axis=1)


In [10]:
df_v3 = reduce_mem_usage(df_v3)

In [11]:
rolling_features = [
        'ask_size',
        # 'bid_size',
        # 'imbalance_with_flag',
        # 'liquidity_imbalance',
        # 'matched_imbalance',
        # 'reference_price_ask_price_imbalance',
        # 'reference_price_bid_price_imbalance',
        # 'reference_price_wap_imbalance',
        # 'far_price_near_price_imbalance',
        ]
    
df_rolling = group_by_stock[rolling_features].rolling(5).agg(['mean', 'std', 'max', 'min']).reset_index()
stats_cols = [f"{c[1]}_{c[0]}_5" for c in df_rolling.columns[2:]]
df_rolling.columns = ['date_id', 'stock_id'] + stats_cols
df_rolling.set_index('_level_2_5', inplace=True)
df_rolling.drop(columns=['date_id', 'stock_id'], inplace=True)

df_v3 = df.merge(df_rolling, left_index=True, right_index=True, how='left')

In [12]:
rolling_features = [
        'ask_size',
        # 'bid_size',
        # 'imbalance_with_flag',
        # 'liquidity_imbalance',
        # 'matched_imbalance',
        # 'reference_price_ask_price_imbalance',
        # 'reference_price_bid_price_imbalance',
        # 'reference_price_wap_imbalance',
        # 'far_price_near_price_imbalance',
        ]
    
df_rolling = group_by_stock[rolling_features].rolling(5).agg(['mean', 'std', 'max', 'min']).reset_index()

In [13]:
stats_cols = [f"{c[1]}_{c[0]}_5" for c in df_rolling.columns[2:]]
df_rolling.columns = ['date_id', 'stock_id'] + stats_cols
df_rolling.set_index('_level_2_5', inplace=True)
df_rolling.drop(columns=['date_id', 'stock_id'], inplace=True)

df_v3 = df.merge(df_rolling, left_index=True, right_index=True, how='left')